In [118]:
import cv2
import numpy as np
from Interval_keypoint import Interval_keypoint

In [119]:
### Constants ###
# Path to the descriptor files
DESCRIPTOR_FILE_PATH = "side_demo_kpt_des"      
# The maximum distance between two matched keypoints
MAX_MATCH_DISTANCE = 50             
# THe nummber of frames in an interval
INTERVAL = 12
# Create a BFMatcher object with Hamming distance (suitable for ORB, BRIEF, etc.)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)            

In [120]:
def load_kpt_des(interval: int) -> list:
    '''
    description: Load the descriptors from the descriptor file.
    param       {int} interval: The timeline interval of the state.
    return      {list}: A list containing the descriptors for the given interval.
    '''
    file_name = (
        "../" + DESCRIPTOR_FILE_PATH + "/" + 
        DESCRIPTOR_FILE_PATH + str(interval) + ".yml"
    )
    file_storage = cv2.FileStorage(file_name, cv2.FILE_STORAGE_READ)
    # Load the descriptors from the file
    descriptors = file_storage.getNode("descriptors").mat()
    file_storage.release()
    return descriptors

In [121]:
def compare_descriptors(descriptors1: np.ndarray, descriptors2: np.ndarray) -> list:
    '''
    description: Compare two sets of descriptors using BFMatcher.
    param       {np.ndarray} descriptors1: Descriptors from the first set.
    param       {np.ndarray} descriptors2: Descriptors from the second set.
    return      {list}: A list containing pairs of matching keypoints' indices (queryIdx, trainIdx).
    '''
    # Match descriptors from two intervals
    raw_matches = bf.match(descriptors1, descriptors2)
    # Extract pairs of matching keypoints' indices from the matches
    matches = [(match.queryIdx, match.trainIdx) for match in raw_matches]
    return matches

In [126]:
debug1 = []
debug2 = []
testdata = [
    [(1, 3), (2, 4), (3, 5), (5, 9)],
    [(3, 4), (4, 5), (8, 9)],
    [(1, 2), (2, 3), (4, 5), (5, 6)],
    [(2, 4), (4, 5), (5, 6), (6, 7)],
    [(1, 1), (2, 3), (3, 5), (4, 6)]
]
for i in range(0, 5):
    matches = testdata[i]
    print("matches: ", matches)
    match_pair_1 = [match[0] for match in matches]
    match_pair_2 = [match[1] for match in matches]
    print("match_pair_1: ", match_pair_1)
    print("match_pair_2: ", match_pair_2)
    last_matches = [x[1] for x in debug1]
    print("last_matches: ", last_matches)
    for j in range(10):
        if j in match_pair_1:
            # Check if the matched keypoint is also matched by another keypoint
            if j in last_matches:
                # If so, update the last match of the keypoint
                debug1[last_matches.index(j)][1] = match_pair_2[match_pair_1.index(j)]
                debug1[last_matches.index(j)][3] = i+2
            else:
                debug1.append([j,match_pair_2[match_pair_1.index(j)],i+1,i+2])
        else:
            if j in last_matches:
                debug2.append(debug1[last_matches.index(j)])
            else:
                # print("j: ", j)
                debug2.append([j,j,1+i,1+i])
    # Remove elements in debug2 from debug1
    # debug1 = [x for x in debug1 if x not in debug2]
    # Add elements in debug1 to debug2
    # print("debug1: ", debug1)
    # print("debug2: ", debug2)

debug2.extend(debug1)
print("Final list of elements: ", debug2)

# Visualize the result in debug2
# Create a black image
img = np.zeros((100, 100, 3), np.uint8)
# Draw the keypoints
for i in range(len(debug2)):
    cv2.circle(img, (debug2[i][0], debug2[i][2]), 1, (255, 0, 0), -1)
    cv2.circle(img, (debug2[i][1], debug2[i][3]), 1, (0, 255, 0), -1)
# Show the image
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

matches:  [(1, 3), (2, 4), (3, 5), (5, 9)]
match_pair_1:  [1, 2, 3, 5]
match_pair_2:  [3, 4, 5, 9]
last_matches:  []
matches:  [(3, 4), (4, 5), (8, 9)]
match_pair_1:  [3, 4, 8]
match_pair_2:  [4, 5, 9]
last_matches:  [3, 4, 5, 9]
matches:  [(1, 2), (2, 3), (4, 5), (5, 6)]
match_pair_1:  [1, 2, 4, 5]
match_pair_2:  [2, 3, 5, 6]
last_matches:  [4, 5, 5, 9, 9]
matches:  [(2, 4), (4, 5), (5, 6), (6, 7)]
match_pair_1:  [2, 4, 5, 6]
match_pair_2:  [4, 5, 6, 7]
last_matches:  [5, 6, 5, 9, 9, 2, 3]
matches:  [(1, 1), (2, 3), (3, 5), (4, 6)]
match_pair_1:  [1, 2, 3, 4]
match_pair_2:  [1, 3, 5, 6]
last_matches:  [6, 7, 5, 9, 9, 4, 3, 5]
Final list of elements:  [[0, 0, 1, 1], [4, 4, 1, 1], [6, 6, 1, 1], [7, 7, 1, 1], [8, 8, 1, 1], [9, 9, 1, 1], [0, 0, 2, 2], [1, 1, 2, 2], [2, 2, 2, 2], [3, 5, 1, 2], [6, 6, 2, 2], [7, 7, 2, 2], [5, 9, 1, 2], [0, 0, 3, 3], [3, 3, 3, 3], [6, 6, 3, 3], [7, 7, 3, 3], [8, 8, 3, 3], [5, 9, 1, 2], [0, 0, 4, 4], [1, 1, 4, 4], [2, 5, 3, 6], [7, 7, 4, 4], [8, 8, 4, 4], [5,